Assamese - word

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

In [ ]:
!pip install emot --upgrade
# to convert emojis to text
!pip install emoji
#to expand a contracted words
!pip install demoji
!pip install contractions

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score
from sklearn.naive_bayes import MultinomialNB
from textblob import TextBlob
import emot
import nltk.data
import numpy as np
import pandas as pd
import re
import time
import string
import emoji
import demoji
import contractions
import nltk
nltk.download('omw-1.4')
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
pd.set_option("max_colwidth" ,220)
wn = nltk.WordNetLemmatizer()
nltk.download('wordnet')
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('english')

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

english_stopwords = stopwords.words("english")
with open('/content/drive/MyDrive/hasoc/task4/assam/assamese.txt', encoding = 'utf-8') as f:
    assam_stopwords = f.readlines()
    for i in range(len(assam_stopwords)):
        assam_stopwords[i] = re.sub('\n','', assam_stopwords[i])
stopwords = english_stopwords + assam_stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
len(stopwords)

295

In [ ]:
demoji.download_codes()
def emo(text):
  temp=emoji.demojize(text,delimiters=(" "," "))
  temp=temp.replace("_","  ")
  return temp

<ipython-input-7-e00aa54160fe>:1: FutureWarning: The demoji.download_codes attribute is deprecated and will be removed from demoji in a future version. It is an unused attribute as emoji codes are now distributed directly with the demoji package.
  demoji.download_codes()


In [ ]:
demoji.download_codes()
def emo(text):
  try:
   temp=emoji.demojize(str(text),delimiters=(" "," "))
   temp=temp.replace("_","  ")
   return temp
  except IndexError:
        return ""

<ipython-input-8-82a5fb4117d1>:1: FutureWarning: The demoji.download_codes attribute is deprecated and will be removed from demoji in a future version. It is an unused attribute as emoji codes are now distributed directly with the demoji package.
  demoji.download_codes()


In [ ]:
punct="!#$%&\'()*+,-.’’/:;<=>?@[\\]^_`{|}~’“‘"

In [ ]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [ ]:
def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', str(text))

In [ ]:
def clean_text(text):
    text = re.sub('@[a-zA-Z0-9]*', '', str(text))#to remove @ and its following word
    text = contractions.fix(text, slang=True)
    text = "".join([word.lower() for word in text if word not in string.punctuation]) #to remove punctuation
    text="".join([word.lower() for word in text if word not in punct])
    text = "".join([word for word in text if not word.isdigit()])#to remove digit
    text = " ".join(word for word in text.split() if word not in stopwords)
    return text

In [ ]:
asa_train=pd.read_csv('/content/drive/MyDrive/hasoc/task4/assam/train_A_AH_HASOC2023.csv')
asa_test=pd.read_csv('/content/drive/MyDrive/hasoc/task4/assam/test_A_AH_HASOC2023.csv')

In [ ]:
asa_train.head()

,S. No.,text,task_1
0,1,লাগিলে ইযাৰ পৰাই কলি লাগাম,NOT
1,2,আমি গেদা মই গেদা হা হা 🤣🤣🤣 ।,HOF
2,3,"মানুহে যেতিয়া নিজৰ জীৱন টো নিজৰ কাৰণে, সমাজৰ কাৰণে, দেশৰ কাৰণে, পৃথিৱীৰ কাৰণে কিমান মূল্যবান বুজি নাপায় তেতিয়া এনেকুৱা হয়",NOT
3,4,কতি খন কোন দিনা দেখা পাও তাকে বাত চায় আছো মই,HOF
4,5,"কাপোৰ খন বাৰু চাইকেল লৈ অহা মানুহজনৰ মুখত দলিয়াব লাগেনে, বেছেৰাটো গুচি গল সেইকাৰণে বেয়া পায়।🤣🤣🤣",NOT


In [ ]:
asa_train['task_1'].value_counts()

HOF    2347
NOT    1689
Name: task_1, dtype: int64

In [ ]:
asa_train['clean_text'] = asa_train['text'].apply(lambda x:emo(x))
asa_train['clean_text'] = asa_train['clean_text'].apply(lambda x:remove_urls(x))
asa_train['clean_text'] = asa_train['clean_text'].apply(lambda x:clean_text(x))

In [ ]:
asa_train['clean_text']

0                                                                                                                                      লাগিলে ইযাৰ পৰাই কলি লাগাম
1                                                                         আমি গেদা মই গেদা rolling floor laughing rolling floor laughing rolling floor laughing ।
2                                      মানুহে যেতিয়া নিজৰ জীৱন টো নিজৰ কাৰণে সমাজৰ কাৰণে দেশৰ কাৰণে পৃথিৱীৰ কাৰণে কিমান মূল্যবান বুজি নাপায় তেতিয়া এনেকুৱা হয়
3                                                                                                                   কতি খন কোন দিনা দেখা পাও তাকে বাত চায় আছো মই
4       কাপোৰ খন চাইকেল অহা মানুহজনৰ মুখত দলিয়াব লাগেনে বেছেৰাটো গুচি গল সেইকাৰণে বেয়া পায়। rolling floor laughing rolling floor laughing rolling floor laughing
                                                                                  ...                                                                            
4031                        

In [ ]:
asa_test['clean_text'] = asa_test['text'].apply(lambda x:emo(x))
asa_test['clean_text'] = asa_test['clean_text'].apply(lambda x:remove_urls(x))
asa_test['clean_text'] = asa_test['clean_text'].apply(lambda x:clean_text(x))

In [ ]:
asa_test['clean_text']

0                                               তথাপিও ভোট বিজেপিকেই দিয়া হব সিধা কথা লাগিলে ভিক্ষাৰি হম তথাপিও ভোট বিজেপিকেই দিম
1                                                                                                মদ খাই নিজৰ কেৰিয়াৰ ধ্বংস কৰিছে।
2                                                            বিপ্লৱৰ কাইঁটীয়া পথ ইমান সহজ নহয় বাপুহঁত।সহজলভ্য জীৱন লাগে তোমালোকক
3                                                                     ক্লা ফকটীয়া ফাল্টু ঘোঁ‌ৰা চহিচ । কাম নাই ষ্টেচনত ঝাড়ু মাৰ গৈ
4       যিয়ে কোচবিহাৰ ৰাজ্যৰ বিৰোধিতা কৰে সি মোগলৰ বংশধৰ কোচবিহাৰ ৰাজ্য হোৱাৰ লগে লগে বস্তুাত ভৰাই ব্ৰহ্মপুত্ৰ নদীত পেলাই দিয়া হব।
                                                                   ...                                                            
1004                                                                                            ভাই বিয়া চিয়া পাতি আমাক পাহৰি গৈছা
1005                                                     সুদুৰ ভাই কোবাট মুৰি আচেলি

In [ ]:
asa_test.head()

,S. No.,text,clean_text
0,1,"তথাপিও ভোট বিজেপিকেই দিয়া হ'ব সিধা কথা লাগিলে ভিক্ষাৰি হ'ম তথাপিও ভোট বিজেপিকেই দিম,,",তথাপিও ভোট বিজেপিকেই দিয়া হব সিধা কথা লাগিলে ভিক্ষাৰি হম তথাপিও ভোট বিজেপিকেই দিম
1,2,মদ খাই নিজৰ কেৰিয়াৰ ধ্বংস কৰিছে।,মদ খাই নিজৰ কেৰিয়াৰ ধ্বংস কৰিছে।
2,3,বিপ্লৱৰ কাইঁটীয়া পথ ইমান সহজ নহয় ঐ বাপুহঁত।সহজলভ্য জীৱন লাগে তোমালোকক????????,বিপ্লৱৰ কাইঁটীয়া পথ ইমান সহজ নহয় বাপুহঁত।সহজলভ্য জীৱন লাগে তোমালোকক
3,4,"ক্লা , ঐ ফকটীয়া ফাল্টু ঘোঁ‌ৰা চহিচ । কাম নাই যদি ষ্টেচনত ঝাড়ু মাৰ গৈ",ক্লা ফকটীয়া ফাল্টু ঘোঁ‌ৰা চহিচ । কাম নাই ষ্টেচনত ঝাড়ু মাৰ গৈ
4,5,যিয়ে কোচবিহাৰ ৰাজ্যৰ বিৰোধিতা কৰে সি মোগলৰ বংশধৰ আৰু কোচবিহাৰ ৰাজ্য হোৱাৰ লগে লগে বস্তুাত ভৰাই ব্ৰহ্মপুত্ৰ নদীত পেলাই দিয়া হব।,যিয়ে কোচবিহাৰ ৰাজ্যৰ বিৰোধিতা কৰে সি মোগলৰ বংশধৰ কোচবিহাৰ ৰাজ্য হোৱাৰ লগে লগে বস্তুাত ভৰাই ব্ৰহ্মপুত্ৰ নদীত পেলাই দিয়া হব।


In [ ]:
X_train=asa_train['clean_text']
X_test=asa_test['clean_text']
y_train=asa_train['task_1']

In [ ]:
X_train.shape, X_test.shape

((4036,), (1009,))

Feature Extraction using word  

In [ ]:
#feature extraction
# word level tf-idf
tfidf_vect2 = TfidfVectorizer(analyzer='word',ngram_range=(1,3))
tfidf_vect2.fit(X_train)
xtrain_tfidf2 =  tfidf_vect2.transform(X_train)
xtest_tfidf2 =  tfidf_vect2.transform(X_test)

In [ ]:
xtrain_tfidf2.shape,xtest_tfidf2.shape

((4036, 29738), (1009, 29738))

In [ ]:
#feature extraction
# word level tf-idf
tfidf_vect3 = TfidfVectorizer(analyzer='word',ngram_range=(1,3))
tfidf_vect3.fit(X_train)
xtrain_tfidf3=  tfidf_vect2.transform(X_train)
xtest_tfidf3=  tfidf_vect2.transform(X_test)

In [ ]:
from sklearn.svm import SVC
svm = SVC()

svm .fit(xtrain_tfidf2, y_train)

y_pred = svm.predict(xtest_tfidf2)

In [ ]:
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression()

logreg .fit(xtrain_tfidf2, y_train)

y_pred = logreg .predict(xtest_tfidf2)


In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.ensemble import VotingClassifier
from sklearn.svm import LinearSVC

clf1 = LogisticRegression(random_state=1)
clf2 = BernoulliNB()
clf3 = LinearSVC(penalty='l2',C=1.0)

eclf = VotingClassifier(estimators=[('LR', clf1), ('BNB', clf2),
('SVC', clf3)], voting='hard')
eclf = eclf.fit(xtrain_tfidf3, y_train)
y_pred_df= eclf.predict(xtest_tfidf3)
print(y_pred_df)

['NOT' 'HOF' 'NOT' ... 'HOF' 'HOF' 'HOF']


In [ ]:
#Submission
# to create .csv file consisting of tweet ids and predicted labels
y_pred_df= pd.DataFrame(data=y_pred, columns=['y_pred'])
Submisssion_assamese = pd.DataFrame()
Submisssion_assamese['S. No.'] = asa_test['S. No.']
Submisssion_assamese['task_1'] = y_pred_df
Submisssion_assamese.to_csv('/content/drive/MyDrive/MUCS_test_assamese_run_4.csv',index = None)